In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family = font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

## (1) 데이터 출처

- 동별 자동차등록대수 : 
    - 서울특별시_행정동별 자동차 등록대수 현황(공공데이터포털) : 2022 기준
        - https://www.data.go.kr/data/15100168/fileData.do
        
    - 15~21년 행정동별 연료별 자동차 등록현황.xlsx
        - https://data.seoul.go.kr/dataList/OA-21236/F/1/datasetView.do#

- 동별 면적 :
    - https://data.seoul.go.kr/dataList/10584/S/2/datasetView.do

## (2) 자동차등록대수

In [79]:
car22= pd.read_csv("../../Data/차,주차장/서울특별시_행정동별 자동차 등록대수 현황_20220430.csv",
           encoding="euc-kr")
car22.head()
car22.info()
car22.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3570 entries, 0 to 3569
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   사용본거지 시군구  3570 non-null   object 
 1   읍면동 (행정동)  3570 non-null   object 
 2   연료         3570 non-null   object 
 3   계          3506 non-null   float64
 4   관용_승용      3570 non-null   int64  
 5   관용_승합      3570 non-null   int64  
 6   관용_화물      3570 non-null   int64  
 7   관용_특수      3570 non-null   int64  
 8   자가용_승용     3570 non-null   int64  
 9   자가용승합      3570 non-null   int64  
 10  자가용화물      3570 non-null   int64  
 11  자가용특수      3570 non-null   int64  
 12  영업용_승용     3570 non-null   int64  
 13  영업용승합      3570 non-null   int64  
 14  영업용화물      3570 non-null   int64  
 15  영업용특수      3570 non-null   int64  
dtypes: float64(1), int64(12), object(3)
memory usage: 446.4+ KB


사용본거지 시군구     0
읍면동 (행정동)     0
연료            0
계            64
관용_승용         0
관용_승합         0
관용_화물         0
관용_특수         0
자가용_승용        0
자가용승합         0
자가용화물         0
자가용특수         0
영업용_승용        0
영업용승합         0
영업용화물         0
영업용특수         0
dtype: int64

In [80]:
del car22["계"]

In [81]:
car22["2022"]=car22.sum(axis=1)

C:\Users\acorn\AppData\Local\Temp\ipykernel_8944\1019585463.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  car22["2022"]=car22.sum(axis=1)


In [85]:
gu=[]
dong=[]
sido=[]

for i in car22["사용본거지 시군구"]:
    sido.append(i.split(" ")[0])

for i in car22["읍면동 (행정동)"]:
    gu.append(i.split(" ")[0])
    dong.append(i.split(" ")[1])
    
car22["시도"]=sido
car22["구별"]=gu
car22["동별"]=dong

In [86]:
car22.loc[car22["구별"] == "영둥포구","구별"]="영등포구"

In [88]:
car22.loc[car22["동별"].str.contains("기타"),"동별"]="기타"

In [89]:
car22["시도"].unique()

array(['서울특별시'], dtype=object)

In [91]:
car22=car22[["구별","동별","2022"]]
car22

,구별,동별,2022
0,종로구,기타,1
1,종로구,기타,2
2,종로구,기타,1
3,종로구,기타,1
4,종로구,기타,1
...,...,...,...
3565,강동구,둔촌2동,377
3566,강동구,둔촌2동,8
3567,강동구,둔촌2동,12
3568,강동구,기타,28


In [92]:
car22_sum=car22.groupby(["구별","동별"])[["2022"]].sum()

In [93]:
car22_sum

2022
구별  동별         
강남구 개포1동   3875
    개포2동  10266
    개포4동   8552
    기타      525
    논현1동  10388
...         ...
중랑구 상봉2동   5353
    신내1동  12815
    신내2동   6891
    중화1동   5571
    중화2동   6185

[475 rows x 1 columns]

In [94]:
car15_21= pd.read_excel("../../Data/회귀분석/15~21년 행정동별 연료별 자동차 등록현황.xlsx",
                        sheet_name=1, skiprows=1)
car15_21.head()
car15_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101561 entries, 0 to 101560
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   연월          101561 non-null  int64 
 1   사용본거지시군구    101561 non-null  object
 2   시군구코드       101561 non-null  int64 
 3   행정동사용본거지    90705 non-null   object
 4   행정동사용본거지코드  101351 non-null  object
 5   용도          101561 non-null  object
 6   차종종별        101561 non-null  object
 7   연료          101561 non-null  object
 8   건수          101561 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 7.0+ MB


In [95]:
car15_21["연도"]= car15_21["연월"].astype(str).str.extract("(20\d{2})")

In [96]:
car15_21.loc[car15_21["행정동사용본거지"].isna(), "행정동사용본거지"] = car15_21["사용본거지시군구"]

In [45]:
# car15_21.info()
# car15_21.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101561 entries, 0 to 101560
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   연월          101561 non-null  int64 
 1   사용본거지시군구    101561 non-null  object
 2   시군구코드       101561 non-null  int64 
 3   행정동사용본거지    101561 non-null  object
 4   행정동사용본거지코드  101351 non-null  object
 5   용도          101561 non-null  object
 6   차종종별        101561 non-null  object
 7   연료          101561 non-null  object
 8   건수          101561 non-null  int64 
 9   연도          101561 non-null  object
dtypes: int64(3), object(7)
memory usage: 7.7+ MB


,연월,사용본거지시군구,시군구코드,행정동사용본거지,행정동사용본거지코드,용도,차종종별,연료,건수,연도
0,201512,서울특별시 종로구,11110,서울특별시 종로구,58000,자가용,화물,경유,1,2015
1,201512,서울특별시 종로구,11110,서울특별시 종로구,60000,자가용,승용,휘발유,2,2015
2,201512,서울특별시 종로구,11110,서울특별시 종로구,60000,자가용,화물,휘발유,1,2015
3,201512,서울특별시 종로구,11110,서울특별시 종로구,61500,자가용,화물,휘발유,2,2015
4,201512,서울특별시 종로구,11110,서울특별시 종로구,62000,자가용,화물,경유,1,2015


In [98]:
sido=[]
gu =[]
dong =[] 

for i in car15_21["행정동사용본거지"]:
    sido.append(i.split(" ")[0])  
    gu.append(i.split(" ")[1])
    try :
        dong.append(i.split(" ")[2])
    except :
        dong.append("기타")
        
car15_21["시도"] = sido
car15_21["구별"] = gu
car15_21["동별"] = dong

In [100]:
car15_21=car15_21[car15_21["시도"]=="서울특별시"]

In [101]:
car15_21["구별"].unique()

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '구로구',
       '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

In [102]:
car15_21["동별"].unique()

array(['기타', '청운효자동', '사직동', '삼청동', '부암동', '무악동', '교남동', '가회동',
       '종로1.2.3.4가동', '종로5.6가동', '이화동', '혜화동', '창신1동', '창신3동', '숭인2동',
       '필동', '소공동', '회현동', '명동', '장충동', '광희동', '을지로3.4.5가동', '을지로동',
       '신당동', '다산동', '약수동', '신당4동', '청구동', '신당5동', '중림동', '후암동', '용산2가동',
       '남영동', '청파동', '원효로1동', '원효로2동', '효창동', '용문동', '한강로동', '이촌2동',
       '이태원1동', '이태원2동', '한남동', '보광동', '평창동', '왕십리2동', '왕십리도선동', '마장동',
       '사근동', '행당1동', '행당2동', '응봉동', '금호1가동', '금호2.3가동', '금호4가동', '창신2동',
       '숭인1동', '옥수동', '성수1가1동', '성수1가2동', '성수2가1동', '성수2가3동', '송정동',
       '용답동', '화양동', '군자동', '중곡1동', '중곡2동', '신당2동', '중곡3동', '중곡4동', '능동',
       '광장동', '신당6동', '동화동', '황학동', '자양1동', '자양2동', '자양3동', '자양4동',
       '구의1동', '구의2동', '구의3동', '이촌1동', '서빙고동', '용신동', '제기동', '전농1동',
       '전농2동', '답십리1동', '답십리2동', '장안1동', '장안2동', '청량리동', '회기동', '휘경1동',
       '휘경2동', '이문1동', '이문2동', '면목4동', '면목5동', '면목본동', '면목7동', '면목3.8동',
       '상봉1동', '상봉2동', '중화1동', '중화2동', '묵1동', '묵2동', '망우본동', '망우3동',
       '신내1동'

In [103]:
car15_21.head()

,연월,사용본거지시군구,시군구코드,행정동사용본거지,행정동사용본거지코드,용도,차종종별,연료,건수,연도,시도,구별,동별
0,201512,서울특별시 종로구,11110,서울특별시 종로구,58000,자가용,화물,경유,1,2015,서울특별시,종로구,기타
1,201512,서울특별시 종로구,11110,서울특별시 종로구,60000,자가용,승용,휘발유,2,2015,서울특별시,종로구,기타
2,201512,서울특별시 종로구,11110,서울특별시 종로구,60000,자가용,화물,휘발유,1,2015,서울특별시,종로구,기타
3,201512,서울특별시 종로구,11110,서울특별시 종로구,61500,자가용,화물,휘발유,2,2015,서울특별시,종로구,기타
4,201512,서울특별시 종로구,11110,서울특별시 종로구,62000,자가용,화물,경유,1,2015,서울특별시,종로구,기타


In [117]:
car15_21=car15_21[["연도","구별","동별","용도","차종종별","연료","건수"]]

In [118]:
car15_21_sum=car15_21.groupby(["구별","동별","연도"])[["건수"]].sum()

In [119]:
car15_21_sum=car15_21_sum.unstack()

In [120]:
car15_21_sum

건수                                                      
연도           2015     2016     2017     2018     2019     2020     2021
구별  동별                                                                 
강남구 개포1동   7313.0   7400.0   7231.0   3773.0   3616.0   3608.0   3643.0
    개포2동   8310.0   6828.0   4950.0   4746.0   7733.0   7963.0  10116.0
    개포4동   7552.0   6514.0   6462.0   6492.0   6267.0   7930.0   8562.0
    기타      531.0    676.0    788.0    675.0    611.0    582.0    531.0
    논현1동  12235.0  13625.0  11037.0  10886.0  10583.0  10447.0  10455.0
...           ...      ...      ...      ...      ...      ...      ...
중랑구 상봉2동   5127.0   5264.0   5178.0   5362.0   5368.0   5379.0   5363.0
    신내1동  12884.0  12815.0  12939.0  12890.0  12949.0  12846.0  12820.0
    신내2동   6902.0   7086.0   7152.0   6950.0   6920.0   6871.0   6894.0
    중화1동   5324.0   5489.0   5718.0   5779.0   5625.0   5585.0   5570.0
    중화2동   6517.0   6424.0   6721.0   6804.0   6467.0   6244.0   6160.0

[478 rows x 7 columns]

In [121]:
car15_21_sum.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 478 entries, ('강남구', '개포1동') to ('중랑구', '중화2동')
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   (건수, 2015)  471 non-null    float64
 1   (건수, 2016)  470 non-null    float64
 2   (건수, 2017)  470 non-null    float64
 3   (건수, 2018)  469 non-null    float64
 4   (건수, 2019)  469 non-null    float64
 5   (건수, 2020)  474 non-null    float64
 6   (건수, 2021)  474 non-null    float64
dtypes: float64(7)
memory usage: 31.4+ KB


In [58]:
car22_sum.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 475 entries, ('강남구', '개포1동') to ('중랑구', '중화2동')
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2022    475 non-null    int64
dtypes: int64(1)
memory usage: 9.0+ KB


In [157]:
df=pd.merge(car15_21_sum, car22_sum, on=["구별","동별"], how="outer")

C:\Users\acorn\AppData\Local\Temp\ipykernel_8944\2541952535.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  df=pd.merge(car15_21_sum, car22_sum, on=["구별","동별"], how="outer")


In [158]:
del df[("건수", "2015")]
del df[("건수", "2016")]
del df[("건수", "2017")]

In [159]:
df.columns=[2018,2019,2020,2021,2022]

In [162]:
df=df.T.stack(["구별","동별"])

In [163]:
df

      구별   동별  
2018  강남구  개포1동     3773.0
           개포2동     4746.0
           개포4동     6492.0
           기타        675.0
           논현1동    10886.0
                    ...   
2022  중랑구  상봉2동     5353.0
           신내1동    12815.0
           신내2동     6891.0
           중화1동     5571.0
           중화2동     6185.0
Length: 2361, dtype: float64

In [164]:
df.to_csv("../../Data/회귀분석/동별차량등록대수.csv", 
            encoding="utf-8")

## (3) 면적

In [32]:
area= pd.read_csv("../../Data/회귀분석/인구밀도_20230424093342.csv")
area.head()

,시점,동별(1),동별(2),동별(3),인구 (명),면적 (㎢),인구밀도 (명/㎢)
0,2018,합계,소계,소계,10049607,605.24,16604
1,2018,합계,종로구,소계,163026,23.91,6817
2,2018,합계,종로구,사직동,9818,1.23,7982
3,2018,합계,종로구,삼청동,3073,1.49,2062
4,2018,합계,종로구,부암동,10537,2.27,4642


In [33]:
area=area[(area["동별(2)"]!="소계") & (area["동별(3)"]!="소계")]

In [34]:
area=area[["시점","동별(2)","동별(3)","면적 (㎢)"]]

In [35]:
area.columns=["연도","구별","동별","면적"]
area.head()

,연도,구별,동별,면적
2,2018,종로구,사직동,1.23
3,2018,종로구,삼청동,1.49
4,2018,종로구,부암동,2.27
5,2018,종로구,평창동,8.87
6,2018,종로구,무악동,0.36


In [36]:
area.to_csv("../../Data/회귀분석/동별면적.csv", 
            encoding="utf-8")